In [7]:
import numpy as np
import math

In [33]:
np.array([[1, 2, 3], [2, 3, 6], [3, 6, 9]]) / np.array([1.0, 2, 3])#[:, np.newaxis]  

array([[ 1. ,  1. ,  1. ],
       [ 2. ,  1.5,  2. ],
       [ 3. ,  3. ,  3. ]])

In [34]:
def fit(X, y, lmbda):
    N, D = X.shape
    cs = np.unique(y)
    C = len(cs)
    sigma2 = np.zeros(D)
    mus = []
    prior = []

    for c in cs:
        i = y == c
        X_c = X[i]
        prior.append(np.mean(i))
        mu_c = np.mean(X_c, axis = 0)
        mus.append(mu_c)
        for x in X_c:
            sigma2 += (x - mu_c) ** 2 / (1.0 * N - C)

    log_prior = np.log(prior)    
    return cs, log_prior, mus, sigma2
        
def predict(model, X):
    cs, log_prior, mus, sigma2 = model
    p = []
    
    for mu in mus:
        p.append((X - mu) / sigma2)
    return
    delta_c = math.log(prior) - np.sum([X[:, j] - mu[c, j] / (2 * sigma2[j]) for j in range(D)])
    X = np.hstack((X, np.ones((len(X), 1))))
    cs, Beta = model
    return np.array([cs[i] for i in np.argmax(X.dot(Beta), axis = 1)])

In [35]:
X = np.array([
    [2, 3, 4, 5],
    [2.2, 3.1, 4.2, 5.3],
    [2.1, 3.2, 4.1, 5.2],
])

y = np.array([1, 1, 2])

In [36]:
model = fit(X, y, 0.1)
predict(model, X)
#np.mean(predict(model, X) == y)